In [ ]:
### test analysis module
import pydrodelta.analysis 
import json
import matplotlib.pyplot as plt


In [ ]:
#### load configuration
bordes_config = json.load(open("pydrodelta_config/bajo_uruguay.json"))


In [ ]:
#### validate configuration
import jsonschema
schema = open("schemas/topology.json")
schema = json.load(schema)


In [ ]:
try:
    jsonschema.validate(instance=bordes_config,schema=schema)
except jsonschema.exceptions.ValidationError as err:
    print(err)
print("Given JSON data is Valid")
#### instantiate boundarySet class from config file (.xml)
topology = pydrodelta.analysis.Topology(bordes_config)
topology


In [ ]:
#### load data
topology.loadData()
### sanitize timeseries
topology.removeOutliers()
topology.detectJumps()


In [ ]:
topology.regularize()


In [ ]:
topology.applyMovingAverage()
topology.applyOffset()


In [ ]:
#### adjust
topology.adjust()

In [ ]:
topology.fillNulls()

In [ ]:
topology.concatenateProno()

In [ ]:
from datetime import timedelta
topology.interpolate(timedelta(days=1))

In [ ]:
topology.saveData("tmp/bajo_uruguay.csv",pivot=True)

In [ ]:
from datetime import timedelta
topology.nodes[0].interpolate(limit=timedelta(hours=24))


In [ ]:
node = topology.nodes[0]
from datetime import timedelta
node.interpolate(limit=timedelta(hours=12))


In [ ]:
data = topology.nodes[1].data
data["valor"]


In [ ]:
data = topology.nodes[0].series[0].data
data["valor"]


In [ ]:
import pandas
topology.nodes[0].data[~pandas.isna(topology.nodes[0].data["valor"])].index.max()

In [ ]:
import pydrodelta.util as util
# int_data = util.interpolateData(data,interpolation_limit=12,tag_column="tag")
data["interpolated"] = data["valor"].interpolate(method='time',limit=48,limit_direction='both',limit_area='inside')

In [ ]:
data["timestart"] = data.index

In [ ]:
data["timestart"].diff()

In [ ]:
import pandas
from datetime import timedelta

node = topology.nodes[1]
node.time_interval = timedelta(hours=1)
data = node.data
# regular = topology.nodes[1].regularize()
data = data.reset_index()

data = data[data["timestart"].dt.hour % 6 == 0][data["timestart"].dt.minute==0]
data["timestart"] = data["timestart"].map(lambda x: x + timedelta(minutes=-15))
data = data.set_index("timestart")


In [ ]:
regular_index = node.createDatetimeIndex()
regular_data = pandas.DataFrame(index=regular_index)
regular_data.index.rename("timestart",True)
time_interval = node.time_interval / 2 # takes half time interval as maximum time distance for interpolation
jdata = regular_data.join(data,how="outer")
jdata = jdata.reset_index()
jdata["diff_with_previous"] = jdata["timestart"].diff()
jdata["diff_with_next"] = jdata["timestart"].diff(periods=-1)
jdata = jdata.set_index("timestart")
jdata["interpolated_backward"] = jdata["valor"].interpolate(method='time',limit=1,limit_direction='backward',limit_area=None)
jdata["interpolated_forward"] = jdata["valor"].interpolate(method='time',limit=1,limit_direction='forward',limit_area=None)
jdata["interpolated_backward_filtered"] = [ x.valor if -x["diff_with_next"] > time_interval else x.interpolated_backward for (i,x) in jdata.iterrows()]
jdata["interpolated_forward_filtered"] = [ x.valor if x["diff_with_previous"] > time_interval else x.interpolated_forward for (i,x) in jdata.iterrows()]
jdata["interpolated_final"] = [x.interpolated_backward_filtered if pandas.isna(x.interpolated_forward_filtered) else x.interpolated_forward_filtered for (i,x) in jdata.iterrows()]
jdata["new_tag"] = [x.tag if pandas.isna(x.interpolated_final) else "interpolated" if pandas.isna(x.valor) else x.tag for (i,x) in jdata.iterrows()]
regular_data = regular_data.join(jdata[["interpolated_final","new_tag"]].rename(columns={"interpolated_final":"valor","new_tag":"tag"}), how = 'left')

In [ ]:
import pydrodelta.util as util
import pandas


In [ ]:
node = topology.nodes[0]
serie = node.series[0]
data = serie.data
column="valor"
tag_column="tag"
df_regular = pandas.DataFrame(index = util.createDatetimeSequence(data.index, node.time_interval, topology.timestart, topology.timeend, topology.time_offset))
df_regular.index.rename('timestart', inplace=True)	 
df_join = df_regular.join(data, how = 'outer')


In [ ]:
def f1(row):
    if -row["diff_with_next"] > timedelta_threshold:
        return row[column]
    else:
        return row["interpolated_backward"]

def f2(row):
    if row["diff_with_previous"] > timedelta_threshold:
        return row[column]
    else:
        return row["interpolated_forward"]

def f3(row):
    if pandas.isna(row["interpolated_forward_filtered"]):
        return row["interpolated_backward_filtered"]
    else:
        return row["interpolated_forward_filtered"]

In [ ]:
    # if interpolate:
    #     # Interpola
    #     min_obs_date, max_obs_date = (df_join[~pandas.isna(df_join[column])].index.min(),df_join[~pandas.isna(df_join[column])].index.max())
    #     df_join["interpolated"] = df_join[column].interpolate(method='time',limit=interpolation_limit,limit_direction='both',limit_area=None if extrapolate else 'inside')
    #     if tag_column is not None:
    #         # print("columns: " + df_join.columns)
    #         df_join[tag_column] = [x[tag_column] if pandas.isna(x["interpolated"]) else "extrapolated" if i < min_obs_date or i > max_obs_date else "interpolated" if pandas.isna(x[column]) else x[tag_column] for (i, x) in df_join.iterrows()]
    #     df_join[column] = df_join["interpolated"]
    #     del df_join["interpolated"]
    #     df_regular = df_regular.join(df_join, how = 'left')
    # else:
timedelta_threshold = node.time_interval / 2 # takes half time interval as maximum time distance for interpolation
df_join = df_join.reset_index()
df_join["diff_with_previous"] = df_join["timestart"].diff()
df_join["diff_with_next"] = df_join["timestart"].diff(periods=-1)
df_join = df_join.set_index("timestart")
df_join["interpolated_backward"] = df_join[column].interpolate(method='time',limit=1,limit_direction='backward',limit_area=None)
df_join["interpolated_forward"] = df_join[column].interpolate(method='time',limit=1,limit_direction='forward',limit_area=None)
df_join["interpolated_backward_filtered"] = df_join.apply(lambda row: f1(row),axis=1)
df_join["interpolated_forward_filtered"] = df_join.apply(lambda row: f2(row),axis=1) #[x[column] if x["diff_with_previous"] > timedelta_threshold else x.interpolated_forward for (i,x) in df_join.iterrows()]
df_join["interpolated_final"] = df_join.apply(lambda row: f3(row),axis=1) # [x.interpolated_backward_filtered if pandas.isna(x.interpolated_forward_filtered) else x.interpolated_forward_filtered for (i,x) in df_join.iterrows()]


In [ ]:
def findex(row):
    return (row.name).isoformat()
data.apply(lambda row: findex(row),axis=1)  

In [ ]:
df_join["interpolated_backward_filtered"] = df_join.apply(lambda row: f1(row), axis=1) 

In [ ]:
df_join["interpolated_final"]

In [ ]:
if tag_column is not None:
    df_join["new_tag"] = [x[tag_column] if pandas.isna(x.interpolated_final) else "interpolated" if pandas.isna(x.valor) else x[tag_column] for (i,x) in df_join.iterrows()]
    df_regular = df_regular.join(df_join[["interpolated_final","new_tag"]].rename(columns={"interpolated_final":column,"new_tag":tag_column}), how = 'left')
else:
    df_regular = df_regular.join(df_join[["interpolated_final",]].rename(columns={"interpolated_final":column}), how = 'left')
# return df_regular

In [ ]:
df_join["interpolated_forward_filtered"]#["valor"]#